In [1]:
#El plan es que en las coordenadas con agua ponemos '0', en las coordenadas con barcos ponemos '1'
# y en las coordenadas donde se ha disparado ponemos 2 si el disparo fue en el agua o 3 si el disparo fue a un barco
# y 4 si el barco está hundido

#self.barcos es un diccionario con llaves la dimension del barco y valor cuanto barcos te quedan

#set_boats: select which boat from the dictionary barcos, select a random orientation (horizontal or vertical), then 
# set the boat at random in the allowed area, considering that position is the head of the boat
#  (the rest of the boat follows either right if orientation==horizontal or down if orientation==vertical)

#the allowed area have zeros, I set with '2' the area close to the boats already set before, so that they don't touch.
#When all the boats are set, I will reset all the '2' with '0'

#barcos = {1 : 4, 2 : 3 , 3 : 2 , 4 : 1}


#set_mask
# I pad the matrix with an extra border, not to be worried if the boat is at the border or not. I set the mask and then I 
# delete the extra padding I put.

#CONSTANTS:
#BOARD_SIZE = (10,10)

import numpy as np

class Board:
    """
    class Board that set a board (a matrix) with boats placed randomly and with random orientation,
    which can be used to play Battleship

    Arguments:
    1) self.player : string
        name of the player
    2) self.board : numpy matrix with integers
        Board notation: 0 -> water
                        1 -> boat
                        2 -> hit on water
                        3 -> hit on boat
                        4 -> boat sunk
    3) self.board_boats : numpy matrix with integers
        This boat is to save what type of boat there is in each coordinate
                        0 -> water (no boat)
                        1 -> 1-dim boat
                        2 -> 2-dim boat
                        3 -> 3-dim boat
                        4 -> 4-dim boat
    3) self.xdim : int
        board first dimension (vertical)
    4) self.ydim : int
        board second dimension (horizontal)
    5) self.boats : dict
        a dictionary with (keys = dimension of the boat, values = how many boats at start)
    6) self.sunk_boats : dict
        a dictionary with (keys = dimension of the boat, values = how many sunk boats)
    Methods:

    1) __init__(self, player : string, board_size : tuple, boats : dict) -> None
        Constructor of the class, it initialize the board with boats placed randomly
    2) set_boat (self, dim_boat : int) -> None
        set a boat of dimension dim_boat with random orientation in a random spot of the board
        without touching other boats
    3) set_mask(self, coord1 : tuple, coord2 : tuple, is_horizontal : bool)) -> None
        take the position of the boat which goes from coord1 to coord2, and set
        all the cells around the boat equal to '2'
    4) polish_board(self) -> None
        Delete all the '2' in the board. Print that the board is ready.
    5) get_board(self) -> numpy matrix
        Return the board
    6) get_board_boats(self) -> numpy matrix
        Return the board with the dimensions of the boats
    7) get_sunk_boats(self) -> dict
        Return the dictionary with the sunk boats
    8) get_shape(self) -> tuple of int
        Return (self.xdim, self.ydim) 
    """
    
    def __init__(self, player, board_size, boats):
        """
        Constructor of the class, it initialize the board with boats placed randomly.
        
        Inputs:
        1) player : string
            name of the player
        2) board_size : tuple of int
            (xdim, ydim) dimension of the board
        3) boats : dict
            a dictionary with (keys = dimension of the boat, values = how many boats)
        """
        self.player = player
        self.board = np.zeros(board_size)
        self.board_boats = np.full(board_size + (3,),(0,0,0))
        self.xdim = board_size[0]
        self.ydim = board_size[1]
        self.boats = boats.copy()
        self.sunk_boats = {key : 0 for key in boats}

        for boat, num in boats.items():
            for j in range(num):
                self.set_boat(boat)
        #reset the boats in the dictionary
        self.boats = boats.copy()
        self.polish_board()


    def set_boat(self, dim_boat):
        """
        set a boat of dimension dim_boat with random orientation in a random spot of the board
        without touching other boats

        Inputs:
        1) dim_boat : int
            dimension of the boat you want to place

        """
        flag = True
        if dim_boat not in self.boats:
            print(f"ERROR: There is no boat with dimension {dim_boat}")
            return None
        if self.boats[dim_boat]>0:
           self.boats[dim_boat]-=1 #substract the boat you already put
        else:
            print(f"Warning: you already put all the boats of dimension {dim_boat}")
            return None
        counter = 0
        while flag and counter<200:
            is_horizontal = np.random.randint(0,2)>0
            if is_horizontal: # horizontal
                coord1 = (np.random.randint(0,self.xdim),np.random.randint(0,self.ydim-dim_boat+1))
                coord2 = (coord1[0], coord1[1] + dim_boat)

                #if all the points where I want to put the boat are free (zeros)
                # +1 is because if not the slicing is empty
                if (self.board[coord1[0],coord1[1]:coord2[1]] == 0).all():
                    self.board[coord1[0],coord1[1]:coord2[1]] = 1
                    for i, coord in enumerate(range(coord1[1],coord2[1])):
                        self.board_boats[coord1[0],coord] = (dim_boat, 0, i)
                    self.set_mask(coord1, coord2, is_horizontal)
                    flag = False
            else: #vertical
                coord1 = (np.random.randint(0,self.xdim-dim_boat+1),np.random.randint(0,self.ydim))
                coord2 = (coord1[0] + dim_boat, coord1[1])

                if (self.board[coord1[0]:coord2[0],coord1[1]]==0).all():
                    self.board[coord1[0]:coord2[0],coord1[1]] = 1
                    for i, coord in enumerate(range(coord1[0],coord2[0])):
                        self.board_boats[coord,coord1[1]] = (dim_boat, 1 ,i)
                    self.set_mask(coord1, coord2, is_horizontal)
                    flag = False
            counter +=1
        if counter >=200:
            print(f"I was not able to")


    def set_mask(self, coord1, coord2, is_horizontal):
        """
        take the position of the boat which goes from coord1 to coord2, and set
        all the cells around the boat equal to '2'

        Inputs:
        1) coord1 : tuple of a pair of int
            coordinate (x,y) of the head of the boat
        2) coord2 : tuple of a pair of int
            coordinate (x,y) of the space after the tail of the boat
        3) is_horizontal : bool
            True : horizontal , False : vertical
        """
        #pad the matrix:
        new_board = np.zeros((self.xdim+2, self.ydim+2))
        new_board[1:-1,1:-1] = self.board
        #add one to coordinates
        coord1 = [c+1 for c in coord1]
        coord2 = [c+1 for c in coord2]

        if is_horizontal: #horizontal
            new_board[coord1[0],coord1[1]-1] = 2
            new_board[coord2[0],coord2[1]] = 2
            new_board[coord1[0]-1, coord1[1]-1:coord2[1]+1] = 2
            new_board[coord1[0]+1, coord1[1]-1:coord2[1]+1] = 2
        else:
            new_board[coord1[0]-1,coord1[1]] = 2
            new_board[coord2[0],coord1[1]] = 2
            new_board[coord1[0]-1:coord2[0]+1, coord1[1]-1] = 2
            new_board[coord1[0]-1:coord2[0]+1, coord1[1]+1] = 2

        #delete padding
        self.board = new_board[1:-1,1:-1]
              
    def polish_board(self):
        """
        Delete all the '2' in the board. Print that the board is ready.
        """
        self.board[np.where(self.board==2)]=0
        #print(f"The board for player {self.player} is ready")

    
    def get_board(self):
        """
        Return the board
        Output:
        1) self.board : numpy matrix
        """
        return self.board
    
    def get_board_boats(self):
        """
        Return the board with boats dimensions
        Output:
        1) self.board : numpy matrix
        """
        return self.board_boats
    
    def get_sunk_boats(self):
        """
        Return the dictionary with the sunk boats
        Output:
        1) self.boats : dict
        """
        dictio = { key : (str(list(self.sunk_boats.values())[i]) + "|" + str(list(self.boats.values())[i])) 
                 for i,key in enumerate(self.boats.keys())}

        return dictio
    
    def add_sunk_boat(self, dim_boat):
        """ 
        Add a sunk boat in the counting of boats
        """
        self.sunk_boats[dim_boat]+=1
    
    def get_shape(self):
        """
        Return the shape of the board
        Output:
        1) (self.xdim, self.ydim) : tuple of int
        """
        return (self.xdim, self.ydim)

 
        



In [2]:
#CONSTANTS

BOARD_SIZE = (10,10)
BOATS = {1 : 4, 2 : 3 , 3 : 2 , 4 : 1}

In [30]:
#functions

def show_rules():
    """ 
    Show the rules of the game
    Output:
    1) bool
      True: player wants to play
      False: player doesn't want to play
    """
    import time
    from IPython.display import HTML


    display(HTML("""<img src="/home/fabrizio/Documenti/DataScience_Course/Battleship_DS23/battleship.jpg"
                width="500" height="300" align="center">"""))

    string = """

    ¿Cómo funciona el juego?
    Vamos a realizar una versión que tiene algunas particularidades respecto al juego original, de manera que sea más sencillo el desarrollo. Veamos cómo funciona:

    1. Hay dos jugadores: tú y la máquina
    2. Un tablero de 10 x 10 posiciones donde irán los barcos
    3. Lo primero que se hace es colocar los barcos. Para este juego los barcos se colocan de manera aleatoria. Ahora bien, puedes empezar colocando los barcos en unas posiciones fijas, que no cambien con cada partida, y después implementarlo aleatoriamente, ya que es más complejo. Los barcos son:
        * 4 barcos de 1 posición de eslora
        * 3 barcos de 2 posiciones de eslora
        * 2 barcos de 3 posiciones de eslora
        * 1 barco de 4 posiciones de eslora

    4. Tanto tú, como la máquina tenéis un tablero con barcos, y se trata de ir "disparando" y hundiendo los del adversario hasta que un jugador se queda sin barcos, y por tanto, pierde.
    5. Funciona por turnos y empiezas tú.
    6. En cada turno disparas a una coordenada (X, Y) del tablero adversario. Si aciertas, te vuelve a tocar. En caso contrario, le toca a la máquina.
    7. En los turnos de la máquina, si acerta también le vuelve a tocar. ¿Dónde dispara la maquina? A un punto aleatorio en tu tablero.
    8. Si se hunden todos los barcos de un jugador, el juego acaba y gana el otro.
    9. Por norma del juego no puede haber barcos pegados el uno con el otro sin espacios entre ellos."""

    for s in string:
        print(s,end="")
        time.sleep(0.03)
    while True:
        ready = input("Are you ready to play? ('yes' or 'no')")
        if ready.lower()=='yes':
            print("Let's start")
            return True
        elif ready.lower()=='no':
            print("Ok, maybe next time!")
            return False
        else: 
            print("You must write either 'yes' or 'no'")

def print_boats(board : Board):
    """ 
    Print how many boats you already sunk
    Input:
    1) board: Board
        an object Board
    """
    boats = board.get_sunk_boats()
    print("List of sunk boats:")
    for key,value in boats.items():
        print(f"{key}-dimensional boat: {np.abs(value)}")
    
def check_victory(board : Board):
    """
    Check the victory condition.
    Inputs:
    1) board : Board
        an object Board
    Outputs:
    1) bool
        True: victory, False: Not yet victory
    """
    return np.array([x!=1 for x in board.get_board()]).all()



def show_board(board : Board):
    """
    Print a nicer visualization of the board
    Inputs:
    1) board : Board
    """
    b = board.get_board()
    tot_string = ""
    new_board = np.full((21,10)," "*5)
    indices_water = (1+2*np.where(b==0)[0], np.where(b==0)[1])
    indices_boat = (1+2*np.where(b==1)[0], np.where(b==1)[1])
    indices_hit = (1+2*np.where((b==2) | (b==3) | (b==4))[0], np.where((b==2) | (b==3) | (b==4))[1])

    #print(indices_water)
    new_board[indices_water] = "| * |"
    new_board[indices_boat] =  "< B >"
    new_board[indices_hit] = "( X )"
    new_board[0::2,:] = "-----"
    #print(new_board)
    for i in range(new_board.shape[0]):
        for j in range(new_board.shape[1]):
            tot_string += new_board[i][j]+" "
        tot_string += "\n"
    return tot_string
    
def show_hits(board : Board):
    """
    Print the board with all the places where the player fired
    Inputs:
    1) board : Board
    """
    tot_string = ""
    shape = board.get_shape()
    b = board.get_board()
    new_board = np.full((shape[0]*2+1,shape[1])," "*5)

    indices_not_hit = (1+2*np.where((b==0) | (b==1))[0], np.where((b==0) | (b==1))[1])
    indices_hit_water = (1+2*np.where(b==2)[0], np.where(b==2)[1])
    indices_hit_boat = (1+2*np.where((b==3))[0], np.where((b==3))[1])
    indices_hit_sunk = (1+2*np.where((b==4))[0], np.where((b==4))[1])

    new_board[indices_not_hit] = "| * |"
    new_board[indices_hit_sunk] =  "< H >"
    new_board[indices_hit_boat] = "( X )"
    new_board[indices_hit_water] = "| O |"
    new_board[0::2,:] = "-----"
    
    print(f"""
    This is the board of {board.player}.
    Legend: ( X ) : hit but not sunk
            ( H ) : hit and sunk
            | * | : not yet fired
            | O | : fired but water
    """)

    for i in range(new_board.shape[0]):
        for j in range(new_board.shape[1]):
            tot_string += new_board[i][j]+" "
        tot_string += "\n"
    return tot_string


def hit(board : Board, coord : tuple):
    """
    Function to fire on a specific coordinate coord.
    Inputs:
    1) coord : tuple of int
        the (x,y) coordinate where you want to fire
    Outputs:
    1) bool
        True if you hit something (water or boat)
        False if you tried to hit a position already hit or out of the board
    2) is_sunk : bool
        True if the boat has been sunk
        False if not
    """
    b = board.get_board()
    shape = board.get_shape()
    is_sunk = False
    if coord[0]>=shape[0] or coord[1]>=shape[1]:
        print(f"ERROR: the point {coord} is out of of the board with dimension {shape}")
        return None, None
    if b[coord[0],coord[1]] not in [0,1]:
        print(f"You already fired the point {coord}, please choose another point")
        return None, None
    elif b[coord[0],coord[1]] == 0:
        print(f"Your attack hit the water at the point {coord}")
        b[coord[0],coord[1]] = 2
        return True, is_sunk
    elif b[coord[0],coord[1]] == 1:
        print(f"You hit a boat at the point {coord}")
        b[coord[0],coord[1]] = 3
        is_sunk = check_if_sunk(board, coord)
        return True, is_sunk
    


def check_if_sunk(board : Board, coord : tuple):
    """
    Function to check if a boat hit at coordinate coord was sunk. It print a message
    explaining if the boat was sunk with the last hit or not.
    Inputs:
    1) coord : tuple of int
        the (x,y) coordinate where you want to fire
    Outputs:
    1) sunk : bool
        True if the boat is sunk
        False if the boat is not sunk
    """
    b = board.get_board()
    b_boats = board.get_board_boats()
    dim_boat = b_boats[coord[0],coord[1]][0]
    orientation = b_boats[coord[0],coord[1]][1]
    index_pos = b_boats[coord[0],coord[1]][2]
    shape = board.get_shape()
    boat_coords = []
    #I call this function only if I hit a boat. If there are no '1' closeby it means the boat is sunk
    

    if dim_boat==1: #1-dim boat is sunk for sure
        sunk = True
        board.add_sunk_boat(dim_boat)
        b[coord[0],coord[1]] = 4 # sunk boat
        print(f"You sank a boat of dimension {dim_boat}!!")

    else:
        if orientation == 0: #horizontal orientation
            for i in range (0,dim_boat):
                boat_coords.append([coord[0],coord[1]-index_pos+i])
        elif orientation == 1: #vertical orientation
            for i in range (0,dim_boat):
                boat_coords.append([coord[0]-index_pos+i,coord[1]])
        if all([b[x[0],x[1]]==3 for x in boat_coords]): #if all pieces of boat are '3' then it's sunk
            for c in boat_coords:
                b[c[0],c[1]] = 4 #sunk boat

            sunk = True
            board.add_sunk_boat(dim_boat)
            print(f"You sank a boat of dimension {dim_boat}!!")
        else:
            print("You didn't sink the boat yet!")
            sunk = False
    return sunk


def get_neighbours(board : Board, coord : tuple):
    """
    Return a list of tuples with the coordinates of the neighbours
    of the cell at the point coord (only horizontal and vertical neighbours)
    """
    shape = board.get_shape()
    tmp = ([(coord[0]+i,coord[1]) for i in [-1,1]] 
                + [(coord[0],coord[1]+i) for i in [-1,1]])
    neighbours = []    
    for x in tmp:
        if x[0] in range(0,shape[0]) and x[1] in range(0,shape[1]):
            neighbours.append(x)
    return neighbours

#Other functions for main!

def player_turn(cpu_board : Board):
    """ 
    Function for the turn of the player
    Inputs:
    1) board : Board
      The Board object of the computer (enemy)
    """
    is_hit = None
    while is_hit==None:
        str = input(f"""Provide the coordinates where you want to fire in the format x,y.
                    Remember the board size is {BOARD_SIZE}""")
        if len(str)!=3:
            print(f"You wrote {str}, you should write x,y where x,y are between 0 and 9 included")            
        else:
            try:
                x = int(str[0])
                y = int(str[2])
            except:
                print(f"x,y must be integer numbers between 0 and 9 included!!")
                continue
            if x not in range(0,10) or y not in range(0,10):
                print(f"x and y must be between 0 and 9 included!")
            else:
                is_hit, _ = hit(cpu_board,(x,y))

    print(show_hits(cpu_board))
    
    return is_hit

In [4]:
show_rules()


  ## Introducción
En este ejercicio vas a crear tu propio juego de **Hundir la flota** en Python. Para el desarrollo del programa neceistarás conocimientos de la librería `numpy`, módulos, bucles, funciones, clases y colecciones d

KeyboardInterrupt: 

In [18]:
#Test for the class Board 
#1) Generate the initial board for one player and for the computer

boats = {1 : 4, 2 : 3 , 3 : 2 , 4 : 1}
my_board = Board("Fabrizio",(10,10),boats)

print(my_board.board)
print("\n\n")

cpu_board = Board("Computer",(10,10),boats)

print(cpu_board.board)


[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 1. 1. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 1. 1. 0. 0.]]



[[0. 0. 0. 0. 1. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 1. 0.]]


In [25]:
hit(my_board,(3,0))

You hit a boat at the point (3, 0)
You didn't sink the boat yet!


(True, False)

In [26]:
print(show_hits(my_board))


    This is the board of Fabrizio.
    Legend: ( X ) : hit but not sunk
            ( H ) : hit and sunk
            | * | : not yet fired
            | O | : fired but water
    
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | | * | | * | < H > | * | | * | | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | < H > | * | | * | < H > | * | | * | | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | | * | | * | | * | | * | | * | | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
( X ) | * | | * | | * | | * | | * | | * | | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | | * | | * | | * | | * | | * | | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | | * | | * | | * | | * | | * | | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | | * | | * | | 

In [31]:
#main code for the game!

import numpy as np
import os

no_victory = True

name = input("Set the name for the player")
print(f"Welcome {name} to this Battleship game!")
rule = input("Do you want me to remind you the rules? ('yes' or 'no')")
while True:
    if rule.lower() =='yes':
        #show_rules()
        break
    elif rule.lower() =='no':
        break
    else:
        print("Please reply either 'yes' or 'no'")

#os.system('cls' if os.name == 'nt' else "printf '\033c'") #this should work on terminal to flush output!

my_board = Board(name, BOARD_SIZE, BOATS)

cpu_board = Board("Computer",BOARD_SIZE, BOATS)

print("""     
This is your board with boats set randomly 
Notation: | * | is water
          < B > is a boat piece
          ( X ) is a spot hit by the enemy
""")
print(show_board(my_board))
#flip a coin to decide who starts
coin_dict = {0 : "head", 1 : "tail"}
player_starts = False
while True:
    coin = input("Choose head or tail")
    if coin.lower() not in ["head", "tail"]:
        print(f"You must write either 'head' or 'tail' but you wrote {coin}")
    else:
        break
if coin_dict[np.random.randint(0,2)]==coin.lower():
    player_starts = True
player_starts = True #comment this!!!!!!!!!
#player starts
while no_victory:
    if player_starts:
        is_hit = True
        while is_hit:
            is_hit = player_turn(cpu_board)

            no_victory = not check_victory(cpu_board)
            print("So far you sank the following boats:")
            print(f"{cpu_board.get_sunk_boats()}")
    
    break



Welcome fabri to this Battleship game!
     
This is your board with boats set randomly 
Notation: | * | is water
          < B > is a boat piece
          ( X ) is a spot hit by the enemy

----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | | * | | * | | * | | * | < B > | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | | * | | * | | * | | * | < B > | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | | * | | * | | * | | * | < B > | * | | * | < B > 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
< B > | * | < B > | * | | * | | * | | * | | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
< B > | * | < B > | * | | * | < B > < B > | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
| * | | * | < B > | * | | * | | * | | * | | * | | * | | * | 
----- ----- ----- ----- ----- ----- ----- ----- ----- ----- 
< B > | * | | * |

In [108]:
input()

ciao


In [125]:
import sys
def delete_last_line():
    "Use this function to delete the last line in the STDOUT"

    #cursor up one line
    sys.stdout.write('\x1b[1A')

    #delete last line
    sys.stdout.write('\x1b[2K')

print("ciaowndunivbu")
sys.stdout.flush()

ciaowndunivbu


In [ ]:
##Save the old set_mask method, this allows touching diagonally the other boats
def set_mask(self, coord1, coord2, is_horizontal):
    """
    take the position of the boat which goes from coord1 to coord2, and set
    all the cells around the boat equal to '2'

    Inputs:
    1) coord1 : tuple of int
        coordinate (x,y) of the head of the boat
    2) coord2 : tuple of int
        coordinate (x,y) of the tail of the boat
    3) is_horizontal : bool
        True : horizontal , False : vertical
    """
    #pad the matrix:
    new_board = np.zeros((self.xdim+2, self.ydim+2))
    new_board[1:-1,1:-1] = self.board
    #add one to coordinates
    coord1 +=1
    coord2 +=1

    if is_horizontal: #horizontal
        new_board[coord1[0],coord1[1]-1] = 2
        new_board[coord2[0],coord2[1]] = 2
        new_board[coord1[0]-1, coord1[1]:coord2[1]] = 2
        new_board[coord1[0]+1, coord1[1]:coord2[1]] = 2
    else:
        new_board[coord1[0]-1,coord1[1]] = 2
        new_board[coord2[0],coord1[1]] = 2
        new_board[coord1[0]:coord2[0], coord1[1]-1] = 2
        new_board[coord1[0]:coord2[0], coord1[1]+1] = 2

    #delete padding
    self.board = new_board[1:-1,1:-1]

In [16]:
dictio1 = {1:4, 2:3, 3:2, 4:1}
dictio2 = {1:0, 2:1, 3:0, 4:1}

num = { key : (str(list(dictio2.values())[i]) + "|" + str(list(dictio1.values())[i])) for i,key in enumerate(dictio1.keys())}
print(num)

{1: '0|4', 2: '1|3', 3: '0|2', 4: '1|1'}


In [41]:
board = np.full((10,10),0)


In [45]:
np.array([x!=0 for x in board]).all()

False